In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'


In [ ]:
from google.colab import files
import pandas as pd
import seaborn as sns
import numpy as np
uploaded = files.upload()

df = pd.read_csv('money_ball.csv')

Saving money_ball.csv to money_ball.csv


In [ ]:
df=df.drop(['Playoffs', 'RankSeason', 'RankPlayoffs'], axis=1)
df['Run_Differential'] = df['RS'] - df['RA']

In [ ]:
target = 'W'
y_train = df[target]
y_test = df[target]

In [ ]:
print('Mean Baseline (using 0 features)')
guess = y_train.mean()
print(guess)

Mean Baseline (using 0 features)
80.90422077922078


In [ ]:
df=df.drop(['OOBP', 'OSLG'], axis=1)

In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Mean Asbsolute Error: {mae} games')

Mean Asbsolute Error: 9.381598920559961 games


In [ ]:
train = df[df['Year'] < 2004]
test = df[df['Year'] > 2003]

In [ ]:
val = train[train['Year'] > 1991]

In [ ]:
train = train[train['Year'] <= 1991]

In [ ]:
train.shape, val.shape, test.shape

((672, 11), (290, 11), (270, 11))

In [ ]:
!pip install category_encoders==2.*

In [ ]:
import numpy as np
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor()
target = 'W'
features = ['G', 'Run_Differential']
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
)

In [ ]:
k = 3
scores = cross_val_score(pipeline, X_val, y_val, cv=k, 
                         scoring='neg_mean_absolute_error')
# print(f'MAE for {k} folds:', -scores)
mae=-scores.mean()
f'Validation Mean Asbsolute Error: {mae:,.0f} games'

'Validation Mean Asbsolute Error: 4 games'

In [ ]:
scores = cross_val_score(pipeline, X_test, y_test, cv=k, 
                         scoring='neg_mean_absolute_error')
# print(f'MAE for {k} folds:', -scores)
mae=-scores.mean()

f'Test Mean Asbsolute Error: {mae:,.0f} games'

'Test Mean Asbsolute Error: 4 games'